In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType, StructField, StructType
from numpy import array
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow import convert_to_tensor
from tensorflow import reshape

2021-12-10 00:36:30.941698: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-10 00:36:30.941723: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
d = 9 #digit to predict

Input (training)

In [3]:
spark = SparkSession.builder.getOrCreate()
df = spark.read.option("header", True).option("inferSchema", True).csv('train.csv')
L = array([list(train.asDict().values()) for train in df.collect()])
del(df)

21/12/10 00:36:35 WARN Utils: Your hostname, Caballero resolves to a loopback address: 127.0.1.1; using 10.0.0.15 instead (on interface wlp0s20f3)
21/12/10 00:36:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/12/10 00:36:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/10 00:36:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/10 00:36:36 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/12/10 00:36:42 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Preprocessing (training)

In [4]:
x_train = L[:,1:785]
y_train = L[:,0]
del(L)
x_train = [x/255. for x in x_train]

In [5]:
x_train = convert_to_tensor(x_train, dtype="float32")
x_train = reshape(x_train, (x_train.shape[0], 28, 28, 1))

2021-12-10 00:36:51.679490: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-10 00:36:51.679514: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-10 00:36:51.679526: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Caballero): /proc/driver/nvidia/version does not exist
2021-12-10 00:36:51.679872: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-10 00:36:51.680581: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 131712000

Training

We will use binary classifiers, where modelD is 1 iff the input is d.

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='tanh'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
y_trainD = [ to_categorical( 1 if y == d else 0 , 2) for y in y_train ]
y_trainD = convert_to_tensor(y_trainD, dtype="float32")
model.fit(x_train, y_trainD, batch_size=10, epochs=10, shuffle=True)
del(y_trainD)

Epoch 1/10
4200/4200 [==============================] - 51s 12ms/step - loss: 0.0523 - accuracy: 0.9829
Epoch 2/10
4200/4200 [==============================] - 51s 12ms/step - loss: 0.0224 - accuracy: 0.9933
Epoch 3/10
4200/4200 [==============================] - 53s 13ms/step - loss: 0.0172 - accuracy: 0.9943
Epoch 4/10
4200/4200 [==============================] - 53s 13ms/step - loss: 0.0156 - accuracy: 0.9950
Epoch 5/10
4200/4200 [==============================] - 51s 12ms/step - loss: 0.0110 - accuracy: 0.9963
Epoch 6/10
4200/4200 [==============================] - 51s 12ms/step - loss: 0.0115 - accuracy: 0.9964
Epoch 7/10
4200/4200 [==============================] - 51s 12ms/step - loss: 0.0094 - accuracy: 0.9970
Epoch 8/10
4200/4200 [==============================] - 51s 12ms/step - loss: 0.0094 - accuracy: 0.9971
Epoch 9/10
4200/4200 [==============================] - 52s 12ms/step - loss: 0.0070 - accuracy: 0.9978
Epoch 10/10
4200/4200 [==============================] - 47s 11m

In [7]:
del(x_train) 

Input (predictions)

In [8]:
df = spark.read.option("header", True).option("inferSchema", True).csv('test.csv')
x_pred = array([list(train.asDict().values()) for train in df.collect()])
del(df)

Preprocessing (predictions)

Output

In [9]:
x_pred = [x/255. for x in x_pred]
x_pred = convert_to_tensor(x_pred, dtype="float32")
x_pred = reshape(x_pred, (x_pred.shape[0], 28, 28, 1))

Prediction

In [10]:
y_pred = model.predict(x_pred, batch_size=10)
del(x_pred)
y_pred = [ y_pred[j][1] for j in range( len(y_pred) ) ]

Output

In [11]:
y_pred = [(j+1, float(y_pred[j])) for j in range(len(y_pred))]

In [12]:
output_df = spark.createDataFrame(spark.sparkContext.parallelize(y_pred),  StructType([
    StructField('ImageId', IntegerType(), True),
    StructField('Probability', FloatType(), True)
]))

In [13]:
output_df.toPandas().to_csv('component'+str(d)+'.csv', index=False)